# CIFAR-10: Training and Testing on a Clean Dataset & Adversarial Robustness Analysis

## Imports and CIFAR-10 loading

In [1]:
# Imports all the module paths
import sys
sys.path.append("../../")

# Requirements for running everything
import torch

# File containing all the required training methods
import defences.cifar10_library as cifar10_library

# For testing
import utils.clean_test as clean_test

# Contains the data loadders
import utils.dataloaders as dataloaders

# For printing outcomes
# import utils.printing as printing

# Example printing, but I removed it to simplify results
# for epsilon in epsilons:
#     printing.print_attack(
#         model,
#         testSetLoader,
#         "FGSM",
#         attacks["FGSM"],
#         epsilon=epsilon,
#     )

Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA
Notebook will use PyTorch Device: CUDA


## Load the dataset

In [2]:
DATA_ROOT = "../../datasets/CIFAR10"

trainSetLoader, _, testSetLoader = dataloaders.get_CIFAR10_data_loaders(
    DATA_ROOT,
    trainSetSize=50000,
    validationSetSize=0,
    batchSize=128,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Save path

In [3]:
SAVE_LOAD_ROOT = "../../data/cifar10"

## Load the Attacks For Training

In [4]:
# A possible attacks array (for nice printing):
import utils.attacking as attacking
import attacks.fgsm as fgsm
import attacks.ifgsm as ifgsm
import attacks.pgd as pgd

# Some attacks use a helper library
import torchattacks

attacks = {}

attacks["FGSM"] = fgsm.fgsm_attack
attacks["I-FGSM"] = ifgsm.ifgsm_attack
attacks["PGD"] = pgd.pgd_attack

## PGD + FGSM Dual Adversarial Training

In [5]:
fgsm_pgd_model = cifar10_library.dual_adversarial_training(
    trainSetLoader,
    attacks["PGD"],
    attacks["FGSM"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_dual_fgsm_pgd",
    epsilon1=(8 / 255),
    epsilon2=0.1,
    alpha=(2 / 255),
    iterations=7,
)

Found already trained model...
... loaded!


In [6]:
clean_test.test_trained_model(fgsm_pgd_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 84.9%


In [7]:
# Save the model
torch.save(fgsm_pgd_model, SAVE_LOAD_ROOT + "/cifar10_dual_fgsm_pgd")

### Evaluation

#### FGSM

In [8]:
# Several values to use for the epsilons
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 84.9%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.01, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 73.21%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


...

... done! Accuracy: 18.42%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.75, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 17.25%
------------------------------------

Testing the model under FGSM Attack using epsilon = 1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 16.71%
------------------------------------



#### PGD

In [9]:
# Several values to use for the epsilons
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

for epsilon in epsilons:
    attacking.attack_model(
        fgsm_pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 84.9%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.01, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 71.95%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


...

... done! Accuracy: 0.87%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.75, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.31%
------------------------------------

Testing the model under PGD Attack using epsilon = 1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.08%
------------------------------------



#### $CW_2$

In [10]:
step_nums = [15, 30, 50, 100]
cs = [0.05, 0.1, 0.3, 1]

for c in cs:
    for step_num in step_nums:
        cw_attack = torchattacks.CW(fgsm_pgd_model, c=c, steps=step_num)
        attacks["CW"] = cw_attack

        attacking.attack_model(
        fgsm_pgd_model,
        testSetLoader,
        "CW",
        attacks["CW"],
        library=True,
        )

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 71.17%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 61.56%
------------------------------------

Testing the model under CW Attack...


...

... done! Accuracy: 12.97%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 7.65%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 4.73%
------------------------------------



## PGD + CW Dual Adversarial Training

In [12]:
# Pass none since if there is no attack function specified, will use CW2 instead
pgd_cw_model = cifar10_library.framework_training(
    trainSetLoader,
    attack_function1=attacks["PGD"],
    attack_function2=None,
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_dual_pgd_cw",
    epsilon1=(8 / 255),
    alpha=(2 / 255),
    iterations=7,
    steps=30,
    c=0.1,
)

Found already trained model...
... loaded!


In [13]:
clean_test.test_trained_model(pgd_cw_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 86.03%


In [14]:
# Save the model
torch.save(pgd_cw_model, SAVE_LOAD_ROOT + "/cifar_dual_pgd_cw")

### Evaluation

#### FGSM

In [15]:
# Several values to use for the epsilons
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        pgd_cw_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 86.03%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.01, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 74.4%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


...

... done! Accuracy: 19.13%
------------------------------------

Testing the model under FGSM Attack using epsilon = 0.75, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 18.59%
------------------------------------

Testing the model under FGSM Attack using epsilon = 1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 18.37%
------------------------------------



#### PGD

In [16]:
# Several values to use for the epsilons
epsilons = [0, 0.01, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

for epsilon in epsilons:
    attacking.attack_model(
        pgd_cw_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 86.03%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.01, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 73.26%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


...

... done! Accuracy: 0.36%
------------------------------------

Testing the model under PGD Attack using epsilon = 0.75, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.25%
------------------------------------

Testing the model under PGD Attack using epsilon = 1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.24%
------------------------------------



#### $CW_2$

In [17]:
step_nums = [15, 30, 50, 100]
cs = [0.05, 0.1, 0.3, 1]

for c in cs:
    for step_num in step_nums:
        cw_attack = torchattacks.CW(pgd_cw_model, c=c, steps=step_num)
        attacks["CW"] = cw_attack

        attacking.attack_model(
        pgd_cw_model,
        testSetLoader,
        "CW",
        attacks["CW"],
        library=True,
        )

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 74.78%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 70.6%
------------------------------------

Testing the model under CW Attack...


...

... done! Accuracy: 11.02%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 6.55%
------------------------------------

Testing the model under CW Attack...


CW Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 4.12%
------------------------------------

